As part of this notebook, we are going to build a **Gmail manager** using the relevanceai SDK! Before we get started, you might wonder - why should you care? 

The answer lies in the simplicity and UX of the platform. While you could [easily do this](https://relevanceai.com/academy) on the RelevanceAI platform itself, I wanted to showcase another way using just pure Python.

In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import dotenv 
from relevanceai import RelevanceAI
from tqdm.auto import tqdm

dotenv.load_dotenv()

True

First, let's create the RelevanceAI client - this client allows us to interact with the platform, and we will be able to perform the following tasks:

1. Create AI Agents/sub-agents
2. Create tools (and connect tools to agents)
3. Trigger tasks 
4. Get results (either in SDK/or in the RelevanceAI platform itself)

You can create an API key, by scrolling to Integrations page - and clicking on "**Create new secret key**".

![](../assets/images/api_key.png)

After creating the API key, I added it to my `.env` file with keys:

```python
{
    "RAI_API_KEY": <api_key>
    "RAI_REGION": <region>
    "RAI_PROJECT": <project>
}
```

Now, we can simply initialise our `client`, it automatically knows to look for these environment variables which got loaded using `dotenv`.

In [3]:
relevanceai = RelevanceAI()

To test that the initialization worked, we could list agents in this account! So, let's do that!

In [4]:
relevanceai.agents.list_agents()

[]

I get an empty list - this means, there are no agents in my account so far. Let's create our "**Gmail manager**".

In [5]:
agent = relevanceai.agents.create_agent(name="Gmail Manager")
agent

Agent(agent_id="1582365a-f8fa-4d20-b867-76eabcdf85ff", name="Gmail Manager")

By default, the relevance platform creates an agent with `gpt-4o-mini`, to use any other provider, we could just pass in another model in the `model` parameter!

![](../assets/images/gmail_manager_agent.png)

Currently, this is an empty agent - without any instructions or knowledge of what it is really supposed to do. So, let's give it some core instructions! 

In [6]:
instructions = """You are my personal Gmail manager whose goal is to summarise my emails!

You will be provided with the complete content of any new emails that land in my inbox. 

Your task is to summarise them with proper headings, follow the below format for summarization:

# Subject: <subject from the email (verbatim)>
## Key requests
- Main requests/key highlights of the email as bullet points
## Call to action
- Any action items from my end as bullet points

You must strictly adhere to the format.
"""

Now that we have defined the instructions, let's add them to our agent!

In [7]:
agent.update_core_instructions(instructions)

{'agent_id': '1582365a-f8fa-4d20-b867-76eabcdf85ff'}

![](../assets/images/core_instructions.png)

As you can see, now the core instructions have been added to the agent! Great! 

In [8]:
url = relevanceai.oauth.add_google_integration(auto_open=True)

Running the above command opens a new tab in your browser - follow the instructions to allow RelevanceAI to have access to your Gmail account. Be rest assured, your private data is secure with us!

![](../assets/images/integration.png)

Now that RelevanceAI has access to the Gmail account, to allow the agent to work on the Gmail, we want to setup "triggers" on the AI agent.

**What are triggers?**

In simple words, triggers are events that "trigger" the agent - any time a new event has occured. For the Gmail manager, this event is anytime a new email is received on the account. To read more about triggers, you can refer to our documentation [here](https://relevanceai.com/docs/get-started/quickstart/integrations#triggers).

So, now let's setup the trigger on the agent!

In [9]:
active_integrations = relevanceai.oauth.list_active_integrations()
active_integrations

ActiveIntegrations(results=[Integration(provider='google, email='meetaiwithaman@gmail.com'])

In the above, given we only have the active Gmail integration in our Relevance platform, there we have one "active integration"! From this integration, we will need the token etc to add a trigger to the AI agent. We can get the complete `google_integration_object` by simply using `getattr` on the `active_integrations` object.

In [10]:
google_integration_object = getattr(active_integrations, "google")
google_integration_object

Integration(provider='google, email='meetaiwithaman@gmail.com'

Now, to add the following as trigger to our agent, we can pass in the complete `google_integration_object` object - and use the `add_google_trigger` method on the agent!

In [11]:
agent.add_google_trigger(google_integration_object)

{'result': 'Success!'}

And now, this agent is active in the RelevanceAI platform and get's triggered any time a new email is received!

![](../assets/images/gmail_trigger.png)

We could also add more tools and create an end to end agent that can also send out emails! More on that, in our next tutorial!